# Fairness in Biometrics: a figure of merit to assess biometric verification system

## The role of alpha

The hyper-parameter $\alpha$ in equation 7 has a crucial role in the computation of $\text{FDR}(\tau)$.
Such value is a business/application decision. For instance, a bank that deploys a biometric verification system in an ATM might prefer to favor fairness towards False Non-Matches and, for this reason, $\alpha$ can assume low values.
On the other hand, in a border control scenario, where false matches are more critical, decision-makers might decide to favor fairness towards False-Matches.
Hence, $\alpha$ should be high.

This example shows how alpha plays the FMR and FNMR trade-offs by observing the FDR values for the synthetiv unfair and fair biometric verification systems.
FDR is defined as:

$$FDR(\tau) = 1- (\alpha A(\tau) + (1-\alpha) B(\tau))$$


In [4]:
%matplotlib widget

import plots
import experiments
import scoring
import fairness_metric
import numpy as np
np.random.seed(10)


import matplotlib.pyplot as plt


#################################################
## PLAYGROUND: PLAY WITH THE VALUES OF VARIABLES BELOW TO CHANGE THE BEHAVIOUR OF THE FAIR VERIFICATION SYSTEM

ALPHA_RANGE = [0, 0.25, 0.5, 0.75, 1]


# RANGE OF DECISION THRESHOLDS USED IN THE DEVELOPMENT SET
TEN_POWER_OF_X = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] # BASICALLY, THE FMR THRESHOLDS

# NUMBER OF IDENTITIES PER DEMOGRAPHIC
IDENTITIES_PER_DEMOGRAPHIC = 300

# REPRESENTS HOW A SAMPLES ARE SAMPLED
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE AVERAGE EUCLIDEAN DISTANCE BETWEEN EACH IDENTITY
# EACH IDENTITY IS SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_MEAN_DISTANCE_FAIR = [1, 1, 1]
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE STANDARD DEVIATIONS OF THE SAMPLES SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_STD_FAIR = [0.2, 0.2, 0.2]


#DEMOGRAPHIC_MEAN_DISTANCE_UNFAIR = [0.8, 1.3, 1.5]
DEMOGRAPHIC_MEAN_DISTANCE_UNFAIR = [0.8, 1.5, 2.3]
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE STANDARD DEVIATIONS OF THE SAMPLES SAMPLED IN THE CORNER OF A HYPERCUBE
#DEMOGRAPHIC_STD_UNFAIR = [0.1, 0.2, 0.4]
DEMOGRAPHIC_STD_UNFAIR = [0.1, 0.2, 0.3]

#################################################


#impostors_per_demographic_dev_fair, genuines_per_demographic_dev_fair, \
#impostors_per_demographic_eval_fair, genuines_per_demographic_eval_fair, n_demographics = experiments.fair_biometric_experiment()
#thresholds_raw_fair = scoring.compute_thresholds(impostors_per_demographic_dev_fair, FAR_THRESHOLDS)

impostors_per_demographic_dev_fair, genuines_per_demographic_dev_fair, \
impostors_per_demographic_eval_fair, genuines_per_demographic_eval_fair,_ = experiments.generate_abstract_experiment(
    DEMOGRAPHIC_DISPLACEMENTS = DEMOGRAPHIC_MEAN_DISTANCE_FAIR,
    DEMOGRAPHIC_STD = DEMOGRAPHIC_STD_FAIR,    
    FAR_THRESHOLDS=TEN_POWER_OF_X,
    DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC=IDENTITIES_PER_DEMOGRAPHIC
)
thresholds_raw_fair = scoring.compute_thresholds(impostors_per_demographic_dev_fair, TEN_POWER_OF_X)


impostors_per_demographic_dev_unfair, genuines_per_demographic_dev_unfair, \
impostors_per_demographic_eval_unfair, genuines_per_demographic_eval_unfair, _ = \
experiments.generate_abstract_experiment(DEMOGRAPHIC_DISPLACEMENTS=DEMOGRAPHIC_MEAN_DISTANCE_UNFAIR,
                                        DEMOGRAPHIC_STD=DEMOGRAPHIC_STD_UNFAIR,
                                        DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC=IDENTITIES_PER_DEMOGRAPHIC)

thresholds_raw_unfair = scoring.compute_thresholds(impostors_per_demographic_dev_unfair, TEN_POWER_OF_X)



fprs_per_alpha_fair = [fairness_metric.compute_fairness_biometric_verification(impostors_per_demographic_eval_fair,
                                                                  genuines_per_demographic_eval_fair, 
                                                                  thresholds_raw_fair, 
                                                                  len(TEN_POWER_OF_X),
                                                                  n_demographics=3,
                                                                  alpha=alpha)
                  for alpha in ALPHA_RANGE
                 ]
    
fprs_per_alpha_unfair = [fairness_metric.compute_fairness_biometric_verification(impostors_per_demographic_eval_unfair,
                                                                  genuines_per_demographic_eval_unfair, 
                                                                  thresholds_raw_unfair, 
                                                                  len(TEN_POWER_OF_X),
                                                                  n_demographics=3,
                                                                  alpha=alpha)
                  for alpha in ALPHA_RANGE
                 ]    
    
    
    
def fairness_plot_raw_per_alpha(fprs_fair, fprs_unfair , x, alphas):
    
    transform = lambda x : -1*np.log10(x)
    colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown","tab:pink", \
              "tab:gray", "tab:olive", "tab:cyan"]
    colors = colors+colors
    
    
    fig, ax = plt.subplots()
    for i, alpha in enumerate(alphas):
        
        ax.plot(transform(x), fprs_fair[i], label=f"$\\alpha={alpha}$", color=colors[i])
        ax.scatter(transform(x),fprs_fair[i], color=colors[i])
        
        ax.plot(transform(x),fprs_unfair[i], color=colors[i], linestyle='dashed')        
        ax.scatter(transform(x),fprs_unfair[i], color=colors[i], linestyle='dashed')


    plt.ylabel('$FDR(\\tau)$', fontsize=18)
    plt.xlabel('x', fontsize=18)
    
    plt.xticks(np.arange(len(x)+1), [""]+[f"$10^{i}$" for i in range(1, len(x)+1)])
    plt.xlim(0.95,len(x)+0.05)
    plt.grid(True)
    
    plt.legend()
    plt.show()  
    

fairness_plot_raw_per_alpha(fprs_per_alpha_fair,fprs_per_alpha_unfair, TEN_POWER_OF_X, ALPHA_RANGE)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …